In [1]:
import os
import cv2
import sys
from zipfile import ZipFile
from urllib.request import urlretrieve
from matplotlib import pyplot as plt
import numpy as np
import dlib


# Base de dados de imagens (exemplo com 2 imagens)
base_de_dados = [
    cv2.imread('myllena.jpg'), 
 #   cv2.imread('agnaldo.jpg'), 
]


# Caminho para o diretório com as imagens das pessoas autorizadas
diretorio_base = './base/'

# Inicialize o detector de faces do dlib
detector = dlib.get_frontal_face_detector()

# Inicialize o modelo de pontos de referência faciais do dlib
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # Baixe o modelo apropriado

# Carregue as imagens das pessoas autorizadas
imagens_autorizadas = []
for filename in os.listdir(diretorio_base):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img = cv2.imread(os.path.join(diretorio_base, filename))
        if img is not None:
            imagens_autorizadas.append(img)
        else:
            print(f"Falha ao carregar a imagem: {filename}")

            

def verificar_acesso(imagem_detectada):    
    # Verifique se a imagem detectada não está vazia
    if imagem_detectada is None:
        return False
  
    # Verifique se as coordenadas do retângulo estão dentro dos limites da imagem
    if (x_left_bottom < 0 or y_left_bottom < 0 or x_right_top >= imagem_detectada.shape[1] or y_right_top >= imagem_detectada.shape[0]):
        return False

    # Converta a região do retângulo para escala de cinza
    imagem_roi = imagem_detectada[y_left_bottom:y_right_top, x_left_bottom:x_right_top]
    imagem_gray = cv2.cvtColor(imagem_roi, cv2.COLOR_BGR2GRAY)

    # Detecte as faces na região da imagem em escala de cinza
    faces = detector(imagem_gray)
   

    for face in faces:
        # Obtenha os pontos de referência faciais
        landmarks = np.array(predictor(imagem_gray, face))

        # Verifique se a imagem da face detectada corresponde a alguma das imagens autorizadas
        for img_autorizada in imagens_autorizadas:
            # Converta a imagem autorizada para escala de cinza
            img_autorizada_gray = cv2.cvtColor(img_autorizada, cv2.COLOR_BGR2GRAY)

            # Detecte as faces na imagem autorizada
            faces_autorizadas = detector(img_autorizada_gray)
            
            for face_autorizada in faces_autorizadas:
                # Obtenha os pontos de referência faciais da imagem autorizada
                landmarks_autorizados = np.array(predictor(img_autorizada_gray, face_autorizada))
                
                # Compare os pontos de referência faciais das duas faces
                diff = np.linalg.norm(landmarks - landmarks_autorizados)
              
                # Defina um limite para considerar como correspondência (ajuste conforme necessário)
                limite_correspondencia = 25.0
                
                if diff < limite_correspondencia:
                    return True  # Se houver correspondência, retorna True

    return False  # Se não houver correspondência com nenhuma imagem autorizada, retorna False




# ========================-Downloading Assets-========================
def download_and_unzip(url, save_path):
    print(f"Downloading and extracting assests....", end="")

    # Downloading zip file using urllib package.
    urlretrieve(url, save_path)

    try:
        # Extracting zip file using the zipfile package.
        with ZipFile(save_path) as z:
            # Extract ZIP file contents in the same directory.
            z.extractall(os.path.split(save_path)[0])

        print("Done")

    except Exception as e:
        print("\nInvalid file.", e)


URL = r"https://www.dropbox.com/s/efitgt363ada95a/opencv_bootcamp_assets_12.zip?dl=1"

asset_zip_path = os.path.join(os.getcwd(), f"opencv_bootcamp_assets_12.zip")

# Download if assest ZIP does not exists.
if not os.path.exists(asset_zip_path):
    download_and_unzip(URL, asset_zip_path)
# ====================================================================

camera = "https://192.168.15.157:8080/video"
""" s = 0
if len(sys.argv) > 1:
    s = sys.argv[1] """

source = cv2.VideoCapture(camera)

win_name = "Camera Preview"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL) 



# Tente abrir a câmera virtual do OBS
source = cv2.VideoCapture(camera)



net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000_fp16.caffemodel")
# Model parameters
in_width = 300
in_height = 300
mean = [104, 117, 123]
conf_threshold = 0.7

while cv2.waitKey(1) != 27:
    has_frame, frame = source.read()
    if not has_frame:
        break
    frame = cv2.flip(frame, 1)
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Create a 4D blob from a frame.
    blob = cv2.dnn.blobFromImage(frame, 1.0, (in_width, in_height), mean, swapRB=False, crop=False)
    # Run a model
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x_left_bottom = int(detections[0, 0, i, 3] * frame_width)
            y_left_bottom = int(detections[0, 0, i, 4] * frame_height)
            x_right_top = int(detections[0, 0, i, 5] * frame_width)
            y_right_top = int(detections[0, 0, i, 6] * frame_height)

            # Verifique o acesso com base na imagem detectada
            imagem_pessoa = frame[y_left_bottom:y_right_top, x_left_bottom:x_right_top]
            acesso_autorizado = verificar_acesso(imagem_pessoa)
            
           

            if acesso_autorizado:
                cv2.rectangle(frame, (x_left_bottom, y_left_bottom), (x_right_top, y_right_top), (0, 255, 0))
                label = "Acesso autorizado"
            else:
                cv2.rectangle(frame, (x_left_bottom, y_left_bottom), (x_right_top, y_right_top), (0, 0, 255))
                label = "Acesso não autorizado"

            # Desenhe a mensagem na imagem
            label_size, base_line = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(
                frame,
                (x_left_bottom, y_left_bottom - label_size[1]),
                (x_left_bottom + label_size[0], y_left_bottom + base_line),
                (255, 255, 255),
                cv2.FILLED,
            )
            cv2.putText(frame, label, (x_left_bottom, y_left_bottom), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    t, _ = net.getPerfProfile()
    label = "Inference time: %.2f ms" % (t * 1000.0 / cv2.getTickFrequency())
    cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
    cv2.imshow(win_name, frame)

source.release()
cv2.destroyWindow(win_name)

TypeError: verificar_acesso() takes 1 positional argument but 5 were given